In [ ]:
%pip install --quiet -U langchain_openai

In [ ]:
import os
import sys
from pathlib import Path
from loguru import logger
import datetime
import tempfile
from langchain_openai import AzureChatOpenAI
sys.path.append('../src') #needed to get the azure config imports to run
from config import LOCAL_CONFIG_DIR, run_azure_config

run_azure_config(LOCAL_CONFIG_DIR)

In [ ]:
now = str(datetime.date.today())
temp_dir_path = tempfile.mkdtemp(prefix=now)
log_file_name = "quickstart.ipynb.log" # only for notebooks
# log_file_name = Path(__file__) + ".log" # only for scripts
log_file_path = Path(temp_dir_path) / log_file_name  #appends automatically if file exists

logger.info(f"created {temp_dir_path=}")
log_level = "DEBUG"
log_format = "<green>{time:YYYY-MM-DD HH:mm:ss.SSS zz}</green> | <level>{level: <8}</level> | <yellow>Line {line: >4} ({file}):</yellow> <b>{message}</b>"
logger.add(sys.stderr, level=log_level, format=log_format, colorize=True, backtrace=True, diagnose=True)
logger.add(log_file_path, level=log_level, format=log_format, colorize=False, backtrace=True, diagnose=True)

In [22]:
use_gpt_4 = input("Use GPT 4? (y or y)").lower()
if use_gpt_4 == "y":
    model_name = os.getenv("MODEL_NAME_GPT")
    deployment_name = os.getenv(
            "AZURE_OPENAI_API_DEPLOYMENT_NAME_GPT"
            )
    logger.info("using gpt 4")
else:
    model_name = os.getenv("MODEL_NAME_GPT35")
    deployment_name = os.getenv(
            "AZURE_OPENAI_API_DEPLOYMENT_NAME_GPT35")
    logger.info("using gpt 3.5")

2024-04-07 14:46:22.378 | INFO     | __main__:<module>:12 - using gpt 3.5
2024-04-07 14:46:22.378 EDT | INFO     | Line   12 (3346147229.py): using gpt 3.5


In [23]:
    llm = AzureChatOpenAI(
        temperature=0.05,
        streaming=True,
        model_name=model_name,
        azure_deployment=deployment_name,
        azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
        openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
        request_timeout=120,
        verbose=False,
    )

In [21]:
os.environ["AZURE_OPENAI_API_ENDPOINT"],os.environ["AZURE_OPENAI_API_VERSION"]

('https://opnai-genai-dataanlytics-sb.openai.azure.com/', '2024-03-01-preview')

In [28]:
from typing import List

from langchain_core.messages import BaseMessage, HumanMessage
from langchain_openai import ChatOpenAI
from langgraph.graph import END, MessageGraph
# from functools import partial #https://mathspp.com/blog/functools-partial

graph = MessageGraph()

def invoke_model(state: List[BaseMessage]):
    return llm.invoke(state)

graph.add_node("oracle", invoke_model)
graph.add_edge("oracle", END)

graph.set_entry_point("oracle")

runnable = graph.compile()

In [29]:
runnable.invoke(HumanMessage("What is 1 + 1?"))

[HumanMessage(content='What is 1 + 1?', id='ca31b179-5e50-4900-b376-957c6d8d0f3d'),
 AIMessage(content='1 + 1 equals 2.', response_metadata={'finish_reason': 'stop'}, id='run-9df335e3-b39b-43df-a7f3-5d83f163e409-0')]

In [30]:
import json

from langchain_core.messages import ToolMessage
from langchain_core.tools import tool
from langchain_core.utils.function_calling import convert_to_openai_tool


@tool
def multiply(first_number: int, second_number: int):
    """Multiplies two numbers together."""
    return first_number * second_number

llm = AzureChatOpenAI(
    temperature=0.05,
    streaming=True,
    model_name=model_name,
    azure_deployment=deployment_name,
    azure_endpoint=os.environ["AZURE_OPENAI_API_ENDPOINT"],
    openai_api_version=os.environ["AZURE_OPENAI_API_VERSION"],
    request_timeout=120,
    verbose=False,
)
model_with_tools = llm.bind(tools=[convert_to_openai_tool(multiply)])

graph = MessageGraph()

def invoke_model(state: List[BaseMessage]):
    return model_with_tools.invoke(state)

graph.add_node("oracle", invoke_model)

def invoke_tool(state: List[BaseMessage]):
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    multiply_call = None

    for tool_call in tool_calls:
        if tool_call.get("function").get("name") == "multiply":
            multiply_call = tool_call

    if multiply_call is None:
        raise Exception("No adder input found.")

    res = multiply.invoke(
        json.loads(multiply_call.get("function").get("arguments"))
    )

    return ToolMessage(
        tool_call_id=multiply_call.get("id"),
        content=res
    )

graph.add_node("multiply", invoke_tool)

graph.add_edge("multiply", END)

graph.set_entry_point("oracle")

In [31]:
def router(state: List[BaseMessage]):
    tool_calls = state[-1].additional_kwargs.get("tool_calls", [])
    if len(tool_calls):
        return "multiply"
    else:
        return "end"

graph.add_conditional_edges("oracle", router, {
    "multiply": "multiply",
    "end": END,
})

In [32]:
runnable = graph.compile()

runnable.invoke(HumanMessage("What is 12 * 13?"))

[HumanMessage(content='What is 12 * 13?', id='82fd79af-2ddb-414d-b5cf-ae91b6f40947'),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'index': 0, 'id': 'call_v6iZ2PORYvejeNxqBMFWYXyi', 'function': {'arguments': '{"first_number":12,"second_number":13}', 'name': 'multiply'}, 'type': 'function'}]}, response_metadata={'finish_reason': 'tool_calls'}, id='run-5c0d34e6-253b-4a11-8187-291854fc74ec-0'),
 ToolMessage(content='156', id='adc13087-650f-4695-bf4e-b6daa170b5ac', tool_call_id='call_v6iZ2PORYvejeNxqBMFWYXyi')]

In [33]:
runnable.invoke(HumanMessage("What is your name?"))

[HumanMessage(content='What is your name?', id='ddcd82ac-83b7-4502-ab07-fbd85b1e419d'),
 AIMessage(content="I'm an AI language model and don't have a personal name, but you can call me Assistant! How can I assist you today?", response_metadata={'finish_reason': 'stop'}, id='run-e6983f83-4a26-4595-a2da-8b93566efc06-0')]